In [1]:
import pandas as pd

df = pd.read_excel(r'C:\Users\Ashton Internship\Desktop\Chat With Excel\Operator List_processed.xlsx')

df.head()

,Region,Country,Operator,Remarks,country_code,MCC/MNC,Direct/Sponsored,handset_display_1,handset_display_2,network_tie_up,prefered_list
0,ASIA & MIDDLE EAST,Afghanistan,Afghanistan Wireless Communication (AWCC),NaN,93,41201,Direct,AF AWCC,AWCC,2G/3G/LTE,Preferred 1
1,EUROPE,Andorra,S.T.A (MOBILAND),NaN,376,21303,Direct,ANDORRA-MOBILAND,M-AND,2G/3G/LTE/VOLTE,Preferred 1
2,CENTRAL & LATIN AMERICA,Anguilla,Cable & Wireless (West Indies) Ltd. Anguilla,NaN,1-264,365840,Direct,FLOW,FLOW,2G/3G/LTE,Preferred 2
3,CENTRAL & LATIN AMERICA,Anguilla,Mossel (Jamaica) Ltd (Digicel) - NETWORK ESTEN...,NaN,1-264,338050/338880,Direct,Digicel,Digicel,2G/3G/LTE,Preferred 1
4,CENTRAL & LATIN AMERICA,Antigua and Barbuda,Digicel antigua e barbuda - NETWORK ESTENSION ...,NaN,1-268,338884,Direct,Digicel,digicel,2G/3G/LTE,Preferred 1


In [2]:
# We can check out the length of the dataframe
len(df)


281

In [4]:
# # Save the modified DataFrame back to the Excel file
# df = df.to_excel(r'C:\Users\Ashton Internship\Desktop\Chat With Excel\Operator List_processed.xlsx', index=False)

In [4]:
import sqlite3

# Create a connection to a local database (This will create a file called mydatabase.db)
db = sqlite3.connect('test.db')

# Copy the dataframe to our SQLite database
df.to_sql('operator_list', db, if_exists='replace')

281

In [83]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///test.db")
print(db.dialect)
print(db.get_usable_table_names())


sqlite
['operator_list']


In [51]:
from langchain_ollama import ChatOllama

# llm = ChatOllama(model = "deepseek-r1", temperature = 0.2,num_gpu = 1,verbose = False)
llm = ChatOllama(model = "llama3.2", temperature = 0.2,num_gpu = 1,verbose = False)
# llm = ChatOllama(model = "mistral", temperature = 0.2,num_gpu = 1,verbose = False)

In [7]:
def getDatabaseSchema():
    return db.get_table_info() if db else "Please connect to database"

In [82]:
# print(getDatabaseSchema())

In [64]:
schema = getDatabaseSchema()

In [76]:
from langchain_core.prompts import ChatPromptTemplate
spell_check_template = """You are an expert spelling checker and AI assistant. Your task is to correct any misspelled region or country names in the user's query and return the corrected query.

### **Instructions:**  
- Correct the following common misspellings in the query:  
  - "IROPE" → "Europe"  
  - "ASIAH" → "Asia"  
  - "OCEANEA" → "Oceania"  
  - "AFRICAH" → "Africa"  
  - "AMERICAH" → "America"  
  - "ANTARTICA" → "Antarctica"  
- If the region is not recognized (e.g., "Mars"), keep it as-is.  
- Return **the entire corrected query** and nothing else—no explanations, no comments.  

### **Examples:**  
**Input:** List all operators in IROPE.  
**Output:** List all operators in Europe.  

**Input:** How many customers are from ASIAH?  
**Output:** How many customers are from Asia?  

**Input:** Show operators in Mars.  
**Output:** Show operators in Mars.  

### **Your Turn:**  
**Input:** {query}  
**Output:**  
"""

spell_check_prompt = ChatPromptTemplate.from_template(spell_check_template)

In [81]:
sql_template = """You are an expert in generating SQL queries. Below is the schema of a MySQL database. Read the schema carefully, paying attention to table names, column names, and case sensitivity.

Your task is to generate a syntactically correct SQL query that answers the user's question based on the given schema. **Only provide the SQL query and nothing else**—no explanations, no comments.

### **Schema:**  
{schema}

### **Instructions:**  
- Use **only** the provided tables and columns in the schema.  
- Be **mindful of case sensitivity** for table and column names.  
- If the user doesn't specify a limit, do **not** fetch excessive records.  
- Do **not** generate queries for non-existent tables or columns.  
- The region/country in the question has already been spell-checked. Use it as-is.  
- **Handle case sensitivity for the `Region` column:**  
  - Convert the region/country value in the query to **UPPERCASE** to match the database format.  
  - For example, if the user asks about "Europe", the query should use `WHERE Region = 'EUROPE'`.  

### **Examples:**  
**Question:** How many albums are in the database?  
**SQL Query:** SELECT COUNT(*) FROM album;

**Question:** How many customers are from Brazil?  
**SQL Query:** SELECT COUNT(*) FROM customer WHERE country = 'Brazil';

**Question:** List all operators in Europe.  
**SQL Query:** SELECT Operator FROM operator_list WHERE Region = 'EUROPE';

### **Your Turn:**  
**Question:** {question}  
**SQL Query:**  
"""


sql_prompt = ChatPromptTemplate.from_template(sql_template)

In [78]:
response_template = """You are an AI assistant skilled in SQL and natural language processing. Your task is to generate **a user-friendly response** based on a SQL query result.

Below is the schema of a MySQL database. Read the schema carefully to understand the structure, including table and column names.

### **Schema:**  
{schema}

### **Instructions:**  
- Respond in **natural, conversational language** while keeping the response **concise and informative**.  
- Do **not** include the SQL query or any technical details in the response.  
- If the result is empty (`[]` or `None`), respond accordingly (e.g., "There are no operators in Europe in the database.").  
- Start the response with a brief sentence that dynamically adapts to the question (e.g., "The operators in Europe are:" or "The total number of customers in Brazil is:").  
- Format the response in a clean, structured way:  
  - For lists, use bullet points (`-`) or numbered lists.  
  - For counts or single values, provide a concise sentence.  
- If the question implies ordering (e.g., "list all operators in alphabetical order"), ensure the response reflects the ordered results.  

### **Examples:**  

**Question:** How many albums are in the database?  
**SQL Query:** SELECT COUNT(*) FROM album;
**Result:** `[(34,)]`  
**Response:** There are 34 albums in the database.  

**Question:** How many customers are from India?  
**SQL Query:** `ELECT COUNT(*) FROM customer WHERE country = 'India';
**Result:** `[(4,)]`  
**Response:** There are 4 customers from India in the database.  

**Question:** List all operators in Europe in alphabetical order.  
**SQL Query:** SELECT Operator FROM operator_list WHERE Region = 'EUROPE' ORDER BY Operator ASC;
**Result:** `[("Operator A",), ("Operator B",), ...]`  
**Response:** The operators in Europe are:  
- Operator A  
- Operator B  
- ...  

### **Your Turn:**  
**Question:** {question}  
**SQL Query:** {query}  
**Result:** {result}  
**Response:**  
"""

response_prompt = ChatPromptTemplate.from_template(response_template)

In [79]:
from langchain_core.runnables import RunnablePassthrough

def runChain(question):
    # Step 1: Spell check
    spell_check_chain = spell_check_prompt | llm
    corrected_question = spell_check_chain.invoke({"query": question}).content
    
    # Step 2: Generate SQL query
    sql_chain = sql_prompt | llm
    sql_query = sql_chain.invoke({"question": corrected_question ,"schema": schema}).content
    
    # Step 3: Execute SQL query
    result = db.run(sql_query)
    
    # Step 4: Generate user-friendly response
    response_chain = response_prompt | llm
    response = response_chain.invoke({
        "question": corrected_question,
        "schema": schema,
        "query": sql_query,
        "result": result
    }).content
    
    return response

In [84]:
# Example usage
question = "Name operators in Asia"
response = runChain(question)
print(response)

The operators in Asia are:

• Afghanistan Wireless Communication (AWCC)
• Batelco (BHR MOBILE PLUS)
• J.V.Bakcell (BAKCELL GSM 2000)
• Zain BH (ex MTC Vodafone )
• GrameenPhone Ltd
• TM Int'l (Bangladesh) Ltd (AKTEL)
• BMobile
• UNN
• SMART ( ex LATEZ-HALLO AXIATA)  
• CamGSM (MOBITEL)
• China Telecom
• China Mobile Communication Corp.
• Telemor
• CSL Hong Kong Limited 
• Hutchison Telecom 
• SmarTone (ex SmarTone Mobile Comms. Ltd.-Vodafone)
• INDOSAT (ex Satelindo)
• Hutchinson CP
• PT. Excelcomindo (Excelcom)
• Smartfren
• ZAIN ( ex Atheer )
• Korek telecom
• HOT MOBILE  (ex MIRS) - ADDITIONAL NETWORK of ISRMC
• Pelephone
• Rakuten
• KDDI
• Softbank
• Umniah Mobile Company -
• OOREDOO (formerly National Mobile Telecommunications Co (Wataniya Telecom))
• Mobile Telecommunications Co. (MTCNet)
• KTC (Viva)
• Alfa Telecom 
• Hutchison Telephone (Macau) Company Limited (Hutchison Telecom)
• C.T.M. (TELEMOVEL+ GSM900-Macau)
• Digi Telecommunications Sdn Bhd
• MMSSB
• U Mobile
• Dhivehi R